In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import networkx as nx
from numba import njit
import imageio.v2 as imageio
import glob
from scipy.sparse import lil_matrix

import matplotlib
matplotlib.use('Agg')

In [15]:
def bkg_img():

    plt.figure(figsize=(6, 4))
    bg_img = mpimg.imread('images/map_apple.png')
    ymin, xmin = [32.31, 34.44]
    ymax, xmax = [37.30, 42.77]
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.imshow(bg_img, extent=[xmin, xmax, ymin, ymax], aspect='auto')

def bkg_blank():

    plt.figure(figsize=(6, 4))
    ymin, xmin = [32.31, 34.44]
    ymax, xmax = [37.30, 42.77]
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

 #make colormaps (register for magma_r, viridis_r)

mname = 'viridis_r'
ncolors = 256
color_array = plt.get_cmap(mname)(range(ncolors))

# change alpha values
nlim = 30
color_array[:nlim,-1] = np.linspace(0.0,1.0,nlim)

# create a colormap object
map_object = LinearSegmentedColormap.from_list(name=f'{mname}_alpha',colors=color_array)

# register this new colormap with matplotlib
plt.colormaps.register(cmap=map_object)

# show some example data
#f,ax = plt.subplots()
#h = ax.imshow(np.random.rand(100,100),cmap=map_object)
#plt.colorbar(mappable=h)

#### Import data

In [5]:
f = open('data/populated_places/datafile_oilscore_minorities.dat', "r")
next(f)
content = f.readlines()
f.close()
    
pop = []
x_coord  = []
y_coord  = []
oilscore = []
ethn_groups = []

for line in content:
    temp1 , temp2, temp3, temp4, *temp5 = line.strip().split()
    pop += [float(temp1)]
    x_coord  += [float(temp2)]
    y_coord  += [float(temp3)]
    oilscore += [float(temp4)]
    ethn_groups += [temp5]

x_coord = np.array(x_coord)
y_coord = np.array(y_coord)
pop = np.array(pop)
oilscore = np.array(oilscore)

In [6]:
adj_geom = np.loadtxt('adj_geom.txt')

print(adj_geom)

[[0.    0.016 0.726 ... 1.78  2.237 0.258]
 [0.016 0.    0.71  ... 1.795 2.251 0.27 ]
 [0.726 0.71  0.    ... 2.484 2.931 0.893]
 ...
 [1.78  1.795 2.484 ... 0.    0.466 1.726]
 [2.237 2.251 2.931 ... 0.466 0.    2.191]
 [0.258 0.27  0.893 ... 1.726 2.191 0.   ]]


In [7]:
thr = 0.1
adj_lim = adj_geom.copy()
adj_lim[adj_lim > thr] = 0

In [ ]:
f = open('data/initial/init_occupier.dat', "r")
next(f)
content = f.readlines()
f.close()

init_occ = np.array([line.strip() for line in content])

f = open('data/betweenness_centrality/bce_values.txt', "r")
next(f)
content = f.readlines()
f.close()

bce_vals = np.array([float(line.strip()) for line in content])

f = open('data/closeness_centrality/closeness_values.txt', "r")
next(f)
content = f.readlines()
f.close()

closeness_vals = np.array([float(line.strip()) for line in content])

f = open('data/eigenvector_centrality/eigenvector_values.txt', "r")
next(f)
content = f.readlines()
f.close()

eigenvector_vals = np.array([float(line.strip()) for line in content])

fiedler_values = {}
with open("data/fiedler/fiedler_values.txt") as f:
    next(f)  # skip header
    for line in f:
        group, val = line.strip().split(",")
        fiedler_values[group] = float(val)

print(fiedler_values)

{'state': 0.0005792586450003632, 'sunni': 0.04534464452869004, 'kurds': 0.0016099047796869486}


In [9]:
G_geom = nx.from_numpy_array(adj_lim)
comps=sorted(nx.connected_components(G_geom))
largest_component = comps[0]
G_main = G_geom.subgraph(largest_component)

important cities

In [10]:
cities = {'Damascus' : [36.29 , 33.51],
          'Aleppo'   : [37.10 , 36.12],
          'Homs'     : [36.72 , 34.73],
          'Ar Raqqah': [39.01 , 35.95],
          'Idlib'    : [36.63 , 35.93],
          'Dayr az Zawr' : [40.14 , 35.34],
          'Latakia'  : [35.79 , 35.53],
          'Tartous'  : [35.89 , 34.89]}

In [11]:
damascus_index = 6176
distances_from_damascus = adj_geom[damascus_index]
new_arr = distances_from_damascus[distances_from_damascus != 0.0]
#print(distances_from_damascus[1195])
print(np.argmax(new_arr))
print(distances_from_damascus[np.argmax(new_arr)])

#closeness_vals_nonzero=closeness_vals[closeness_vals != 0.0]
#print(np.argmax(closeness_vals))
#print(closeness_vals[np.argmax(closeness_vals)])

#print(closeness_vals[6177])

dam_norm = (distances_from_damascus - distances_from_damascus.min()) / \
           (distances_from_damascus.max() - distances_from_damascus.min())

print(dam_norm[1274])
#print(dam_norm[np.argmin(dam_norm)])

1274
3.542
1.0


#### Simulation

In [232]:
# parameters

eps_dist = 0.3 #0.3
c_pop = 0.01 #0.01
c_os = 4.21 #4.21
c_di = 0.76 #0.76
c_ethn = 3 #3
c_bce = 80 #80
c_dam=0.1 #0.2
c_closeness= 30 #30/40
c_fiedler=45

In [ ]:
#First iteration algorithm
def charges_calc(occ_state):

    occ_state = np.asarray(occ_state)
    n = np.zeros(len(occ_state), dtype=float)

    for i in range(len(occ_state)):
        match = occ_state[i] in ethn_groups[i]      # True if the occupier matches a dominant ethnic group
        n[i] = 0.01*( pop[i] + 100 )*( 1 + 0.5*(float(match) - 0.5) )

    return n

def battle(I, J, prob, charg, occ_state):

    nI = prob*charg[I]     # attacker strength
    nJ = charg[J]          # defender strength
    diff = abs(nI-nJ)

    charg[I] -= nI         # attacker looses strength they mobilised

    if nI<=nJ:   
        charg[J] -= nI     # defender loses equal strength
    else:
        occ_state[J] = occ_state[I]
        charg[J] = diff  

    return charg, occ_state

def check_structural_collapse(occ_state, charges, loss_threshold, decay_factor):
  
    assad_nodes = np.where(occ_state == "state")[0]
    
    total_centrality = np.sum(eigenvector_vals[assad_nodes] + bce_vals[assad_nodes] + closeness_vals[assad_nodes])
    
    lost_nodes = np.where(occ_state != "state")[0]
    lost_centrality = np.sum(eigenvector_vals[lost_nodes] + bce_vals[lost_nodes] + closeness_vals[lost_nodes])
    
    # Se la perdita supera la soglia → collasso
    if lost_centrality / total_centrality > loss_threshold:
        for node in assad_nodes:
            # Penalizza più i nodi lontani da Damasco
            penalty = decay_factor + 0.5 * distances_from_damascus[node]  # più lontano = più riduzione
            charges[node] *= max(0.2, 1 - penalty)  # non scendere sotto 10%
    
    return charges

def iteration(init_state):
    final_state = init_state.copy()
    charges = charges_calc(init_state)

    half_pi = np.pi / 2

    for i in range(len(init_state)):
        neighbour_indices = np.nonzero(adj_lim[i])[0]

        if neighbour_indices.size == 0:
            continue

        # Get valid target neighbours (different occupier)
        mask_diff = init_state[neighbour_indices] != init_state[i]
        targets = neighbour_indices[mask_diff]

        if targets.size == 0:
            continue

        dam_distances = distances_from_damascus[targets]

        # Vectorised probability calculation
        p_pop = np.arctan(c_pop * pop[targets] + eps_dist) / half_pi
        p_os = 1 - np.arctan(c_os * oilscore[targets]) / half_pi
        p_dis = 1 -  np.arctan(c_di * adj_geom[i, targets]) / half_pi
        p_bce = np.arctan(c_bce * bce_vals[targets]) / half_pi
        p_closeness = np.arctan(c_closeness * closeness_vals[targets]) / half_pi
        p_eigenvector = eigenvector_vals[targets]
        p_damascus = 1 - np.arctan(c_dam*dam_distances) / half_pi

        probs = p_pop * p_os * p_dis * p_damascus
        
        

        # combine with logical ... AND ... OR ... 
        #p_os_dis = p_os + p_dis - p_os*p_dis                            
        #probs = p_pop * p_os_dis 

        # combine with logical ... OR ... AND ... 
        #p_os_dis = p_os * p_dis                             
        #probs = p_pop + p_os_dis - p_pop*p_os_dis

        # combine with logical ... OR ... OR ... 
        #p_os_dis = p_os + p_dis - p_os*p_dis                            
        #probs = p_pop + p_os_dis - p_pop*p_os_dis

        # centrality measures
        probs = probs + p_bce - probs*p_bce
        probs= probs + p_closeness - probs*p_closeness
        probs = probs + p_eigenvector - probs*p_eigenvector

        # Ethnic match bonus
        ethnic_match = np.array( [init_state[i] in ethn_groups[t] for t in targets] )
        probs[ethnic_match] = 1 - (1 - probs[ethnic_match]) / c_ethn

        invade_idx = np.argmax(probs)
        J = targets[invade_idx]

        charges, final_state = battle(i, J, probs[invade_idx], charges, final_state)
    
    #charges = check_structural_collapse(final_state, charges, loss_threshold=3, decay_factor=0.5)


    return final_state

def plot_state_frame(occ_state,I):

    bkg_blank()

    ind_state = np.where(occ_state == "state")[0]
    ind_sunni = np.where(occ_state == "sunni")[0]
    ind_kurds = np.where(occ_state == "kurds")[0]

    x_state, y_state = [x_coord[ind_state],y_coord[ind_state]]
    x_sunni, y_sunni = [x_coord[ind_sunni],y_coord[ind_sunni]]
    x_kurds, y_kurds = [x_coord[ind_kurds],y_coord[ind_kurds]]

    heatmap, xedges, yedges = np.histogram2d(x_state, y_state, bins=300)
    plt.imshow( heatmap.T, origin='lower', cmap='ocean_r', 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 1
    )

    heatmap, xedges, yedges = np.histogram2d(x_sunni, y_sunni, bins=200)
    plt.imshow( heatmap.T, origin='lower', cmap='viridis_r_alpha', 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 1
    )

    heatmap, xedges, yedges = np.histogram2d(x_kurds, y_kurds, bins=200)
    plt.imshow( heatmap.T, origin='lower', cmap='magma_r_alpha', 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 1
    )

    plt.text(
        0.03, 0.9, f"{I+1}",
        transform=plt.gca().transAxes,
        fontsize=14, color='white',
        bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.3')
    )

    for city in cities:
        plt.scatter(cities[city][0], cities[city][1], c='black', s=1, marker='+')

    plt.savefig(f'images/frames/frame_{I+1}.png', bbox_inches = 'tight', dpi=300)
    plt.close()

    return [len(x_state),len(x_sunni),len(x_kurds)]

In [ ]:
#Markov-chain based algorithm
max_fiedler= max(fiedler_values.values())
rng = np.random.default_rng()


def compute_fiedler_values(occ_state, groups=['state', 'sunni', 'kurds']):
    fiedler_vals = {}

    for group in groups:
        group_nodes = np.where(occ_state == group)[0]

        if len(group_nodes) == 0:
            fiedler_vals[group] = 0.0
            continue

        G_group = G_geom.subgraph(group_nodes).copy()

        L_group = nx.laplacian_matrix(G_group).todense()
        eigenvalues_group = np.linalg.eigvalsh(L_group)

        # Takes the second-smallest eigenvalue (Fiedler value) of the Laplacian. 
        # We apply a numerical threshold to filter out values that are effectively zero 
        # due to floating-point precision, ensuring that only genuinely positive eigenvalues 
        # are considered. If none remain, we assign 0.0.
        valid_eig = eigenvalues_group[eigenvalues_group > 3.08e-13]
        fiedler_val = np.min(valid_eig) if len(valid_eig) > 0 else 0.0
        fiedler_vals[group] = fiedler_val

    return fiedler_vals





multipliers_base = {'state':1.0, 'sunni':1.0,'kurds':1.0}
def charges_calc(occ_state, apply_fiedler_init=True, fiedler_init=None):
    occ_state = np.asarray(occ_state)
    n = np.zeros(len(occ_state), dtype=float)

    for i in range(len(occ_state)):
        faction = occ_state[i]
        match = faction in ethn_groups[i] #True if the occupier matches a dominant ethnic group

        base = 0.01 * (pop[i] + 100) * (1 + 0.5 * (float(match) - 0.5)) 

        mul = multipliers_base.get(faction, 1.0) 
        
        if apply_fiedler_init and fiedler_init is not None:
            fval = fiedler_init.get(faction, 0.0)
            fscale = (fval / max_fiedler)  # in [0.5,1]
            mul *= fscale
            
            n[i] = base * mul

        else:  
            n[i] = base 

    return n

def check_structural_collapse(occ_state, charges, loss_threshold, decay_factor):
  
    state_nodes = np.where(occ_state == "state")[0]
    
    total_centrality = np.sum(eigenvector_vals[state_nodes] + bce_vals[state_nodes] + closeness_vals[state_nodes])
    
    lost_nodes = np.where(occ_state != "state")[0]
    lost_centrality = np.sum(eigenvector_vals[lost_nodes] + bce_vals[lost_nodes] + closeness_vals[lost_nodes])
    
    #If the loss exceeds the threshold → collapse
    if lost_centrality / total_centrality > loss_threshold:
        for node in state_nodes:
            #Penalizes more the nodes far from Damascus
            penalty = decay_factor + 0.5 * distances_from_damascus[node]  
            charges[node] *= max(0.2, 0.5 - penalty)  
    
    return charges



def battle(I, J, prob, charg, occ_state, attacker_label):
   # def_factor = dynamic_def_factor(occ_state, attacker_label)
    nI = prob*charg[I]     # attacker strength
    nJ = charg[J]  #* def_factor   # defender strength
    diff = abs(nI-nJ)

    charg[I] -= nI

    if nI<=nJ:   
        charg[J] -= nI     # defender loses equal strength
    else:
        occ_state[J] = occ_state[I]
        charg[J] = diff  

    return charg, occ_state


def iteration(init_state, charges, iter):
    final_state = init_state.copy()
    half_pi = np.pi / 2

    n = len(init_state)
    T = lil_matrix((n, n), dtype=float)   
    planned_attacks = []


    for i in range(n):
        neighbour_indices = np.nonzero(adj_lim[i])[0]
        if neighbour_indices.size == 0:
            T[i, i] = 1.0
            continue

        

        # Get valid target neighbours (different occupier)
        mask_diff = final_state[neighbour_indices] != final_state[i]
        targets = neighbour_indices[mask_diff]
        mask_diff = final_state[neighbour_indices] == final_state[i]
        friends = neighbour_indices[mask_diff]
        if targets.size == 0:
            continue

        dam_distances = distances_from_damascus[targets]

        # Vectorised probability calculation
        p_pop = np.arctan(c_pop * pop[targets] + eps_dist) / half_pi
        p_os = 1 - np.arctan(c_os * oilscore[targets]) / half_pi
        p_dis = 1 -  np.arctan(c_di * adj_geom[i, targets]) / half_pi
        p_bce = np.arctan(c_bce * bce_vals[targets]) / half_pi
        p_closeness = np.arctan(c_closeness * closeness_vals[targets]) / half_pi
        p_eigenvector = eigenvector_vals[targets]
        p_damascus = 1 - np.arctan(c_dam*dam_distances) / half_pi

        # combine with logical ... AND ... AND ...
        probs = p_pop * p_os * p_dis * p_damascus
        
        # combine with logical ... AND ... OR ... 
        #p_os_dis = p_os + p_dis - p_os*p_dis                            
        #probs = p_pop * p_os_dis 

        # combine with logical ... OR ... AND ... 
        #p_os_dis = p_os * p_dis                             
        #probs = p_pop + p_os_dis - p_pop*p_os_dis

        # combine with logical ... OR ... OR ... 
        #p_os_dis = p_os + p_dis - p_os*p_dis                            
        #probs = p_pop + p_os_dis - p_pop*p_os_dis

        # betweeness centrality
        probs = probs + p_bce - probs*p_bce
        probs= probs + p_closeness - probs*p_closeness
        probs = probs + p_eigenvector - probs*p_eigenvector

        
        # Ethnic match bonus
        ethnic_match = np.array( [init_state[i] in ethn_groups[t] for t in targets] )
        if ethnic_match.any():
            probs[ethnic_match] = 1 - (1 - probs[ethnic_match]) / c_ethn
        
        
        dist_neigh = adj_geom[i, friends]
        w_move = np.exp(-c_dam * dist_neigh)
        eig_neigh = eigenvector_vals[friends]
        clo_neigh = closeness_vals[friends]   
        bce_neigh = bce_vals[friends]
        w_move *= (1.0 + 2 * eig_neigh) * (1.0 + 1 * clo_neigh) * (1.0 + 1 * bce_neigh)
        

        s = w_move.sum()
        if s <= 0:
            # fallback: self loop
            T[i, i] = 1.0
        else:
            w_norm = w_move / s
            T[i, friends] = w_norm  #assign row i
        
        #


        p_attack = probs / probs.sum()
        
        #extract a stochastic target J and save the attack 
        J = rng.choice(targets, p=p_attack)
        prob_on_J = probs[targets == J][0]
        planned_attacks.append((i, int(J), float(prob_on_J)))

    T_csr = T.tocsr()
    moved = T_csr.T.dot(charges)    #T^T @ charges
    charges = (1.0 - 0.2) * charges + 0.2 * moved

    for (i, J, p_on_J) in planned_attacks:
        charges, final_state = battle(i, J, p_on_J, charges, final_state, final_state[i])
        charges = check_structural_collapse(final_state, charges, loss_threshold=13, decay_factor=0.2)

    


    return final_state, charges

def plot_state_frame(occ_state,I):

    bkg_blank()

    ind_state = np.where(occ_state == "state")[0]
    ind_sunni = np.where(occ_state == "sunni")[0]
    ind_kurds = np.where(occ_state == "kurds")[0]

    x_state, y_state = [x_coord[ind_state],y_coord[ind_state]]
    x_sunni, y_sunni = [x_coord[ind_sunni],y_coord[ind_sunni]]
    x_kurds, y_kurds = [x_coord[ind_kurds],y_coord[ind_kurds]]

    heatmap, xedges, yedges = np.histogram2d(x_state, y_state, bins=300)
    plt.imshow( heatmap.T, origin='lower', cmap='ocean_r', 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 1
    )

    heatmap, xedges, yedges = np.histogram2d(x_sunni, y_sunni, bins=200)
    plt.imshow( heatmap.T, origin='lower', cmap='viridis_r_alpha', 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 1
    )

    heatmap, xedges, yedges = np.histogram2d(x_kurds, y_kurds, bins=200)
    plt.imshow( heatmap.T, origin='lower', cmap='magma_r_alpha', 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], alpha = 1
    )

    plt.text(
        0.03, 0.9, f"{I+1}",
        transform=plt.gca().transAxes,
        fontsize=14, color='white',
        bbox=dict(facecolor='black', alpha=0.5, boxstyle='round,pad=0.3')
    )

    for city in cities:
        plt.scatter(cities[city][0], cities[city][1], c='black', s=1, marker='+')

    plt.savefig(f'images/frames/frame_{I+1}.png', bbox_inches = 'tight', dpi=300)
    plt.close()

    return [len(x_state),len(x_sunni),len(x_kurds)]

In [ ]:
N = 100
init_state = init_occ.copy()
occ_nums = np.zeros((3,N))

for iter in tqdm(range(N)):

    out_state = iteration(init_state)
    occ_nums[:,iter] = plot_state_frame(out_state,iter)
    init_state = out_state.copy()

In [270]:
N = 100
init_state = init_occ.copy()
fiedler_init = compute_fiedler_values(init_state) 
charges = charges_calc(init_state, apply_fiedler_init=True, fiedler_init=fiedler_init)
occ_nums = np.zeros((3,N))

for iter in tqdm(range(N)):
    new_recruit = charges_calc(init_state, apply_fiedler_init=False)
    charges = 0.9999 * charges + 0.001 * new_recruit

    out_state, charges = iteration(init_state, charges,iter)
    occ_nums[:,iter] = plot_state_frame(out_state,iter)
    init_state = out_state.copy()

100%|██████████| 100/100 [01:45<00:00,  1.06s/it]


In [272]:
frame_files = sorted(glob.glob("images/frames/frame_*.png"), key=lambda f: int(f.split('_')[-1].split('.')[0]))

# Read and save as GIF
frames = [imageio.imread(f) for f in frame_files]
imageio.mimsave("images/animations/animation_test.gif", frames, duration=0.4) 

## TO ADD

Laplacian matrix with Fiedler value //
Recruitment with Markov chain //
Maybe Avalanche process to check collapse //